In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

In [ ]:
import numpy as np
import rioxarray
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, radiometry, orbit, scene

In [ ]:
# product definition
product_urlpath = (
    "data/S1A_IW_SLC__1SDV_20211223T170557_20211223T170624_041139_04E360_B8E2.SAFE"
)
dem_urlpath = "data/Gran-Sasso-3m-DEM-small.tif"
measurement_group = "IW3/VV"
output_urlpath = "Gran-Sasso-10m-RTC-SLC.tif"
output_gtc_urlpath = output_urlpath.replace("RTC", "GTC")
orbit_group = None
calibration_group = None
multilook = None
interp_method = "nearest"
grouping_area_factor = (1.0, 1.0)
open_dem_raster_kwargs = {}
kwargs = {}

In [ ]:
%%time
apps.terrain_correction(
    product_urlpath,
    measurement_group,
    dem_urlpath,
    output_urlpath=output_gtc_urlpath,
)
geocoded_beta0 = rioxarray.open_rasterio(output_gtc_urlpath)
geocoded_beta0.plot(vmin=0, vmax=3)

In [ ]:
apps.terrain_correction(
    product_urlpath,
    measurement_group,
    dem_urlpath,
    correct_radiometry="gamma_bilinear",
    output_urlpath=output_urlpath,
    grouping_area_factor=(1, 5),
)
geocoded_beta0_c = rioxarray.open_rasterio(output_urlpath)
geocoded_beta0_c.plot(vmin=0, vmax=3)

## CHECK INTERNAL FUNCTIONS

### READ ORBIT AND INTERPOLATE

In [ ]:
orbit_group = orbit_group or f"{measurement_group}/orbit"
calibration_group = calibration_group or f"{measurement_group}/calibration"

measurement_ds = xr.open_dataset(product_urlpath, engine="sentinel-1", group=measurement_group, **kwargs)  # type: ignore
measurement = measurement_ds.measurement

dem_raster = scene.open_dem_raster(dem_urlpath, **open_dem_raster_kwargs)

orbit_ecef = xr.open_dataset(product_urlpath, engine="sentinel-1", group=orbit_group, **kwargs)  # type: ignore
position_ecef = orbit_ecef.position
calibration = xr.open_dataset(product_urlpath, engine="sentinel-1", group=calibration_group, **kwargs)  # type: ignore
beta_nought_lut = calibration.betaNought

In [ ]:
dem_ecef = scene.convert_to_dem_ecef(dem_raster)

### BACKWARD GEOCODING DEM and DEM_CENTERS

In [ ]:
acquisition = apps.simulate_acquisition(position_ecef, dem_ecef)

### COMPUTE GAMMA WEIGHTS

In [ ]:
beta_nought = xarray_sentinel.calibrate_intensity(measurement, beta_nought_lut)
beta_nought

In [ ]:
coordinate_conversion = None
if measurement_ds.attrs["sar:product_type"] == "GRD":
    coordinate_conversion = xr.open_dataset(
        product_urlpath,
        engine="sentinel-1",
        group=f"{measurement_group}/coordinate_conversion",
        **kwargs,
    )  # type: ignore
    ground_range = xarray_sentinel.slant_range_time_to_ground_range(
        acquisition.azimuth_time,
        acquisition.slant_range_time,
        coordinate_conversion,
    )
    interp_kwargs = {"ground_range": ground_range}
elif measurement_ds.attrs["sar:product_type"] == "SLC":
    interp_kwargs = {"slant_range_time": acquisition.slant_range_time}
    if measurement_ds.attrs["sar:instrument_mode"] == "IW":
        beta_nought = xarray_sentinel.mosaic_slc_iw(beta_nought)
else:
    raise ValueError(
        f"unsupported sar:product_type {measurement_ds.attrs['sar:product_type']}"
    )

In [ ]:
geocoded = apps.interpolate_measurement(
    beta_nought,
    multilook=multilook,
    azimuth_time=acquisition.azimuth_time,
    interp_method=interp_method,
    **interp_kwargs,
)
geocoded

In [ ]:
grid_parameters = radiometry.azimuth_slant_range_grid(
    measurement_ds, coordinate_conversion, grouping_area_factor
)
grid_parameters

In [ ]:
%%time
weights = radiometry.gamma_weights(
    dem_ecef,
    acquisition,
    **grid_parameters,
)

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=3, figsize=(30, 15))

_ = geocoded.plot(ax=axes[0], vmax=3)
axes[0].grid(c="black")

_ = (geocoded / weights).plot(ax=axes[1], vmax=3)
axes[1].grid(c="black")

_ = weights.plot(ax=axes[2], vmax=3, x="x")
axes[2].grid(c="black")